In [1]:
import pandas as pd 
import numpy as np
import os

In [2]:
print(os.getcwd())

c:\Users\sydwi\Desktop\QSML\data\QS4ML-VU-100\final_data


In [7]:
raw_merge_data_dir = os.getcwd() + '\\cleaned_merge_data'
raw_merge_data_files = ['walk_merge_raw.csv', 'bike_merge_raw.csv', 'run_merge_raw.csv', 'sit_merge_raw.csv', 'syd_merge_raw.csv']
df_list = []
for i in raw_merge_data_files:
    df = pd.read_csv(raw_merge_data_dir + '\\' + i)
    df_list.append(df)

In [8]:
print(raw_merge_data_dir)

c:\Users\sydwi\Desktop\QSML\data\QS4ML-VU-100\final_data\cleaned_merge_data
